In [1]:
import pandas as pd
import numpy as np
import re
from os import listdir

# Data transformation from original file

Sheet names: 
- 'App - Usage Time'
- 'App - Usage Count'
- 'Web - Usage Time'
- 'Web - Usage Count'
- 'Device Unlocks'

In [2]:
raw_data = pd.read_excel('D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/raw/StayFree Export - Total Usage - 6_24_23 (1).xls',
                         sheet_name='Web - Usage Time').rename({'Unnamed: 0': ''}, axis=1)
raw_data.head(3)

,,Device,"October 28, 2022","October 29, 2022","October 30, 2022","October 31, 2022","November 1, 2022","November 2, 2022","November 3, 2022","November 4, 2022",...,"June 16, 2023","June 17, 2023","June 18, 2023","June 19, 2023","June 20, 2023","June 21, 2023","June 22, 2023","June 23, 2023","June 24, 2023",Total Usage
0,1001.tl,motorola moto g(6) plus,0s,0s,0s,0s,0s,0s,0s,0s,...,0s,0s,0s,0s,0s,0s,0s,0s,0s,1s
1,1001tracklists.com,motorola moto g(6) plus,0s,0s,0s,0s,0s,0s,0s,0s,...,0s,0s,0s,0s,0s,0s,0s,0s,0s,27s
2,100daysofcode.com,motorola moto g(6) plus,0s,0s,0s,0s,0s,0s,0s,0s,...,0s,0s,0s,0s,0s,0s,0s,0s,0s,12s


Dataframe transformation

Web files

Three groups will be given by the groupby function: 
- 'Windows Extensión para Chrome'
- 'Windows Extensión para Edge'
- 'motorola moto g(6) plus'

Sheet names: 
- 'App - Usage Time'
- 'App - Usage Count'
- 'Web - Usage Time'
- 'Web - Usage Count'
- 'Device Unlocks'

In [14]:
filename = 'Web - Usage Time'
if 'App' in filename:
    print('Yes, in filename')
elif 'App'
else: print('Not in filename')

Not in filename


In [13]:
import pandas as pd

def read_excel_sheets(file_path):
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names

    dataframes_dict = {}
    
    # Read each sheet and store it in the dictionary
    for sheet_name in sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name).rename({'Unnamed: 0': ''}, axis=1)
        dataframes_dict[sheet_name] = df

    sheet_names_list = list(dataframes_dict.keys())  
    transformed_dataframes = {}

    for sheet_name in sheet_names_list:
        dataframe = dataframes_dict[sheet_name]

        if 'Web' in sheet_name:
            dataframe = dataframe.drop(columns=['Total Usage'])
            dataframe = dataframe.iloc[:-4]
            
            grouped_data = dataframe.groupby('Device')

            # Transformation of every dataframe in the groupby
            for device, group in grouped_data:
                # Transpose the dataframe
                group = group.T
                group.columns = group.iloc[0]

                # Eliminating duplicated columns
                duplicated_columns = group.columns[group.columns.duplicated()]
                group = group.loc[:, ~group.columns.duplicated()]

                # Drop the first row since it is now the header
                group = group[1:].reset_index().rename({'index': 'date'}, axis=1)
                group = group.iloc[1:]

                if 'Time' in sheet_name:
                # Time transformation to minutes, (code made in main_notebook_data_transformation notebook)
                    for col in group.columns:
                        if col != 'date':
                            group[col] = round(pd.to_timedelta(group[col]).dt.total_seconds() / 60, 2)
                        else:
                            pass
                sheet_name_appropiate = sheet_name.lower() \
                                                  .replace('-', '') \
                                                  .replace('  ', ' ') \
                                                  .replace(' ', '_')
                device_name_appropiate = device.split(' ')[-1]
                transformed_dataframes[f'{sheet_name_appropiate}_{device_name_appropiate}'] = group
                        
        elif ('App' in sheet_name) or ('Device' in sheet_name):
            if 'Device' in dataframe.columns:
                # If 'Device' column exists, drop both 'Total Usage' and 'Device'
                dataframe = dataframe.drop(columns=['Total Usage', 'Device'])
            else:
                # If 'Device' column does not exist, only drop 'Total Usage'
                dataframe = dataframe.drop(columns=['Total Usage'])

            dataframe = dataframe.iloc[:-4]

            dataframe = dataframe.T

            dataframe.columns = dataframe.iloc[0]
            duplicated_columns = dataframe.columns[dataframe.columns.duplicated()]
            dataframe = dataframe.loc[:, ~dataframe.columns.duplicated()]

            dataframe = dataframe[1:].reset_index().rename({'index': 'date'}, axis=1)

            # Time transformation to minutes, (only time data, not count data)
            for col in dataframe.columns:
                if col != 'date':
                    dataframe[col] = round(pd.to_timedelta(dataframe[col]).dt.total_seconds() / 60, 2)
                else: pass

            sheet_name_appropiate = sheet_name.lower() \
                                              .replace('-', '') \
                                              .replace('  ', ' ') \
                                              .replace(' ', '_')
            transformed_dataframes[sheet_name_appropiate] = dataframe

    return transformed_dataframes

excel_file_path = "D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/raw/StayFree Export - Total Usage - 6_24_23 (1).xls"
transformed = read_excel_sheets(excel_file_path)

In [2]:
import pandas as pd

def read_excel_sheets(file_path):
    """
    Read different sheets of an Excel file into separate dataframes and apply specific transformations.

    Parameters:
        file_path (str): The path to the Excel file.

    Returns:
        dict: A dictionary containing the transformed dataframes for each sheet.
    """
    
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names

    # Create a dictionary to store the dataframes for each sheet
    dataframes_dict = {}
    
    # Read each sheet and store it in the dictionary
    for sheet_name in sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name).rename({'Unnamed: 0': ''}, axis=1)
        dataframes_dict[sheet_name] = df

    # Get a list of sheet names from the dictionary
    sheet_names_list = list(dataframes_dict.keys())  
    
    transformed_dataframes = {}

    # Process each sheet
    for sheet_name in sheet_names_list:
        dataframe = dataframes_dict[sheet_name]

        if 'Web' in sheet_name:
            # Drop the 'Total Usage' column and remove the last 4 rows
            dataframe = dataframe.drop(columns=['Total Usage'])
            dataframe = dataframe.iloc[:-4]
            
            grouped_data = dataframe.groupby('Device')

            # Transformation of every dataframe in the groupby
            for device, group in grouped_data:
                # Transpose the dataframe and set the first row as column headers
                group = group.T
                group.columns = group.iloc[0]

                # Eliminate duplicated columns in the dataframe
                duplicated_columns = group.columns[group.columns.duplicated()]
                group = group.loc[:, ~group.columns.duplicated()]

                # Drop the first row since it is now the header
                group = group[1:].reset_index().rename({'index': 'date'}, axis=1)
                group = group.iloc[1:]

                if 'Time' in sheet_name:
                    # Time transformation to minutes (code made in main_notebook_data_transformation notebook)
                    for col in group.columns:
                        if col != 'date':
                            group[col] = round(pd.to_timedelta(group[col]).dt.total_seconds() / 60, 2)
                        else:
                            pass

                # Format the sheet name and device name for the transformed dataframe's key
                sheet_name_appropriate = sheet_name.lower() \
                                                  .replace('-', '') \
                                                  .replace('  ', ' ') \
                                                  .replace(' ', '_')
                
                device_name_appropriate = device.split(' ')[-1]
                # Store the transformed dataframe in the final dictionary
                transformed_dataframes[f'{sheet_name_appropriate}_{device_name_appropriate}'] = group
                        
        # For sheets containing 'App' or 'Device', perform different transformations
        elif ('App' in sheet_name) or ('Device' in sheet_name):
            if 'Device' in dataframe.columns:
                # If 'Device' column exists, drop both 'Total Usage' and 'Device'
                dataframe = dataframe.drop(columns=['Total Usage', 'Device'])
            else:
                # If 'Device' column does not exist, only drop 'Total Usage'
                dataframe = dataframe.drop(columns=['Total Usage'])

            dataframe = dataframe.iloc[:-4].T
            dataframe.columns = dataframe.iloc[0]

            duplicated_columns = dataframe.columns[dataframe.columns.duplicated()]
            dataframe = dataframe.loc[:, ~dataframe.columns.duplicated()]

            dataframe = dataframe[1:].reset_index().rename({'index': 'date'}, axis=1)

            # Time transformation to minutes, only for time data, not count data
            if 'Time' in sheet_name:
                
                for col in dataframe.columns:
                    if col != 'date':
                        dataframe[col] = round(pd.to_timedelta(dataframe[col]).dt.total_seconds() / 60, 2)
                    else:
                        pass

            sheet_name_appropriate = sheet_name.lower() \
                                              .replace('-', '') \
                                              .replace('  ', ' ') \
                                              .replace(' ', '_')
            transformed_dataframes[sheet_name_appropriate + '_motorola'] = dataframe

    # Format the sheet names for the transformed dataframe's key
    final_data = {key.lower().replace('plus', 'motorola'): value for key, value in transformed_dataframes.items()}
    return final_data

if __name__ == "__main__":
    # Replace the file_path with the actual path to your Excel file
    excel_file_path = "D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/raw/StayFree Export - Total Usage - 6_24_23 (1).xls"
    transformed = read_excel_sheets(excel_file_path)

    # Save the dataframes into csv files
    path_final_files = "D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/"
    for key, values in transformed.items():
        values.to_csv(path_final_files + key + '.csv', index=None)

In [21]:
transformed.keys()

dict_keys(['app_usage_time', 'app_usage_count', 'web_usage_time_chrome', 'web_usage_time_edge', 'web_usage_time_motorola', 'web_usage_count_chrome', 'web_usage_count_edge', 'web_usage_count_motorola', 'device_unlocks'])

In [71]:
# Drop unnecessary columns
raw_data = raw_data.drop(columns=['Total Usage'])

# Eliminate the last 4 rows of the data
raw_data = raw_data.iloc[:-4]

grouped_data = raw_data.groupby('Device')
transformed_dataframes = {}

# Transformation of every dataframe in the groupby
for device, group in grouped_data:
    # Transpose the dataframe
    group = group.T
    group.columns = group.iloc[0]

    # Eliminating duplicated columns
    duplicated_columns = group.columns[group.columns.duplicated()]
    group = group.loc[:, ~group.columns.duplicated()]

    # Drop the first row since it is now the header
    group = group[1:].reset_index().rename({'index': 'date'}, axis=1)
    group = group.iloc[1:]

    # Time transformation to minutes, (code made in main_notebook_data_transformation notebook)
    for col in group.columns:
        if col != 'date':
            group[col] = round(pd.to_timedelta(group[col]).dt.total_seconds() / 60, 2)
        else:
            pass
        
    transformed_dataframes[device] = group

App files

In [20]:
# Drop unnecesary columns
raw_data = raw_data.drop(columns=['Total Usage', 'Device'])

# Eliminate the last 4 rows of the data
raw_data = raw_data.iloc[:-4]

# Transpose the dataframe
raw_data = raw_data.T

# Set the first row as column names
raw_data.columns = raw_data.iloc[0]

# Eliminating duplicated columns
duplicated_columns = raw_data.columns[raw_data.columns.duplicated()]
raw_data = raw_data.loc[:, ~raw_data.columns.duplicated()]

# Drop the first row since it is now the header
raw_data = raw_data[1:].reset_index().rename({'index': 'date'}, axis=1)

# Time transformation to minutes, (only time data, not count data)
for col in raw_data.columns:
    if col != 'date':
        raw_data[col] = round(pd.to_timedelta(raw_data[col]).dt.total_seconds() / 60, 2)
    else: pass

# Testing

In [66]:
data = pd.read_csv('D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/web_usage_count_motorola.csv',
                   delimiter=';',
                   parse_dates=['date'])
data.head(2)

,date,udea.edu.co,chat.openai.com,tiktok.com,vm.tiktok.com,openai.com,google.com,instagram.com
0,2022-10-28,2,0,2,2,0,12,0
1,2022-10-30,1,0,10,3,0,5,0


Function to filter the dataframe based on the values we want

In [30]:
def transform_dataframe(dataframe: pd.DataFrame, values_greater_than):
    data_copy = dataframe.copy()

    #packing the name of columns and sums in a dictionary to use in further codes
    dictionary_sums = {}
    for column in data_copy:
        if (column != 'date') and \
           (sum(data_copy[column]) >= values_greater_than):
            
            dictionary_sums[column] = sum(data_copy[column])

    names_of_columns_to_stay = [column_name for column_name in dictionary_sums.keys()]
    names_of_columns_to_stay.insert(0, 'date')

    #filtering dataframe
    transformed_dataframe = data_copy[names_of_columns_to_stay]
    
    return transformed_dataframe

# Transformation of the actual data

In [5]:
path = "D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed"
dir_list = listdir(path)
dir_list

['.gitkeep',
 'app_usage_count_motorola.csv',
 'app_usage_time_motorola.csv',
 'device_unlocks.csv',
 'web_usage_count_chrome.csv',
 'web_usage_count_edge.csv',
 'web_usage_count_motorola.csv',
 'web_usage_time_chrome.csv',
 'web_usage_time_edge.csv',
 'web_usage_time_motorola.csv']

If you want to transform files with a specified word in it, replace it in the fist line of the code.
- For "count" files, the delimiter should be changed to ";"
- For "time" files, the delimiter should be changed to ","

In [77]:
word_to_find = "count" # word in the name of the file that you want to search for
for file in dir_list:
    word_is_in = re.search(word_to_find, file)
    if word_is_in:
        file_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/{file}'
        data = pd.read_csv(file_path,
                           delimiter=';',
                           parse_dates=['date'])
        
        data_transformed = transform_dataframe(data, 100)
        data_transformed = data_transformed.set_index('date')
        data_transformed.sort_values(by='date', ascending=True, inplace=True)
        data_transformed = data_transformed[~data_transformed.duplicated()]
        data_transformed = data_transformed.reset_index()

        export_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/{file}'
        data_transformed.to_csv(export_path, 
                                sep=';',
                                index=None,
                                date_format='%Y-%m-%d')

Transformation to 'device_unlocks.csv' file

In [62]:
file_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/device_unlocks.csv'
data = pd.read_csv(file_path,
                   delimiter=';',
                   parse_dates=['date'])

export_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/device_unlocks.csv'
data.to_csv(export_path, 
                        sep=';',
                        index=None,
                        date_format='%Y-%m-%d')

C:\Users\User\AppData\Local\Temp\ipykernel_12776\77304124.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(file_path,
